In [42]:
import language_tool_python
import pymorphy2
import string
import torch
import copy
import json
from zemberek import TurkishSpellChecker, TurkishMorphology

In [49]:
#удаление пробелов перед знаками препинания
def correct_sentence(sentence):
    for punct in string.punctuation:
        if punct in sentence:
            sentence = sentence.replace(' ' + punct, punct)
    return sentence
#исправление грамматических и орфографических ошибок с помощью LanguageTool (локально)
def grammar_correction(text, lang, replaces_dict=None):
    if replaces_dict != None:
        replaces = [elem['new_string'] for elem in replaces_dict if elem['check']]
        replaces_indexes = [[elem['result_start'], elem['result_start'] + len(elem['new_string']) -1] for elem in replaces_dict if elem['check']]
    if lang != 'tr':
        if not lang:
            # tool = language_tool_python.LanguageToolPublicAPI('auto')
            tool = language_tool_python.LanguageTool('auto')
        else:
            # tool = language_tool_python.LanguageToolPublicAPI(lang)
            tool = language_tool_python.LanguageTool(lang)
        # print(tool.check('Ткст для прверки которя'))
        if replaces_dict == None:
            errors = tool.check(text)
            matches = []
            for error in errors:
                if error.replacements is not None and error.offset is not None:
                    error_dict = {'error_word': text[error.offset:error.offset + error.errorLength],
                                'start': error.offset, 'end': error.offset + error.errorLength - 1,
                                'replacements': error.replacements}
                    matches.append(error_dict)
            text_corrected = tool.correct(text)
            grammar_dict = {'corrected_text': text_corrected, 'number_errors': len(errors)}
            if len(errors) > 0:
                grammar_dict['errors'] = matches
        else:
            matches = []
            for i, word in enumerate(replaces):
                if word == text[replaces_indexes[i][0]:replaces_indexes[i][1]]:
                    errors = tool.check(word)
                    for error in errors:
                        if error.replacements is not None and error.offset is not None:
                            error_dict = {'error_word': text[error.offset:error.offset + error.errorLength],
                                        'start': error.offset, 'end': error.offset + error.errorLength - 1,
                                        'replacements': error.replacements}
                        matches.append(error_dict)
                    word_corrected = tool.correct(word)
                    text_corrected[replaces_indexes[i][0]:replaces_indexes[i][1]] == word_corrected
            text_corrected = correct_sentence(text_corrected)
            grammar_dict = {'corrected_text': text_corrected, 'number_errors': len(errors)}
            if len(errors) > 0:
                grammar_dict['errors'] = matches
    else:
        morphology = TurkishMorphology.create_with_defaults()
        spell_checker = TurkishSpellChecker(morphology)
        text_list = text.split()
        text_corrected = []
        number_errors = 0
        matches = []
        for elem in text_list:
            replacements = list(spell_checker.suggest_for_word(elem))
            corrected_word = replacements[0]
            elem_check = elem
            signs = string.punctuation
            for sign in signs:
                if sign in elem_check:
                    elem_check = elem_check.replace(sign, '')
            if elem_check != corrected_word:
                number_errors += 1
                if len(replacements) > 0:
                    error_dict = {
                        'error_word': elem_check,
                        'start': text.find(elem_check), 'end': text.find(elem_check) + len(elem_check) - 1,
                        'replacements': replacements}
                matches.append(error_dict)
            if elem[-1] in string.punctuation:
                text_corrected.append(corrected_word + elem[-1])
            else:
                text_corrected.append(corrected_word)
        text_corrected = ' '.join(text_corrected)
        text_corrected = correct_sentence(text_corrected)
        grammar_dict = {'corrected_text': text_corrected, 'number_errors': number_errors}
        if number_errors > 0:
            grammar_dict['errors'] = matches
    return grammar_dict

In [50]:
text = "İnsanoğlu aslında ne para ne sevgi ne kariyer ne şöhret ne de çevre ile sonsuza dek mutlu olabilecek bir " \
       "yapıya sahiptir. Dış kaynaklardan gelebilecek bu mutluluklar sadece belirli bir zaman için insanı mutlu " \
       "kılıyor. Kişi bu kaynakları elde ettiği zaman belirli bir dönem için kendini iyi hissediyor, ancak alışma " \
       "dönemine girdiği andan itibaren bu iyilik hali hızla tükeniyor. Mutlu olma sanatının özü bu değildir. Gerçek " \
       "mutluluk, kişinin her türlü olaya ve duruma karşı kendini pozitif tutarak mutlu hissedebilmesi halidir. Bu " \
       "davranış şeklini edinen insan, zor günlerde güçlü, mutlu günlerde zevk alan biri olur ve mutluluğu kalıcı " \
       "kılar. "
grammar_correction(text, lang='tr')

TurkishMorphology instance initialized in 12.60196590423584


2023-05-20 17:08:24,011 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 12.60196590423584



{'corrected_text': 'ihsanoğlu aslında de para de sevgi de kariyer de şöhret de da çevre ile sonsuz de mutlu olabilecek bir yapıya sahiptir. dış kaynaklardan gelebilecek bu mutluluklar sadece belirli bir zaman için insan mutlu kılıyor. kişi bu kaynaklar ele ettiği zaman belirli bir dönem için kendini iki hissediyor, ancak çalışma döneminde girdiği anda itibaren bu iyilik ali hızlı tükeniyor. mutlu olmak sanatçının ölü bu değildir. gerek mutluluk, kişinin her türlü olay de durum karşı kendini pozitif tutarak mutlu hissedebilmesi halidir. bu davranış şeklini edilen insan, zor günlerde güçlü, mutlu günlerde sevk olan bir olur de mutluluğu kalıcı kılar.',
 'number_errors': 34,
 'errors': [{'error_word': 'İnsanoğlu',
   'start': 0,
   'end': 8,
   'replacements': ['ihsanoğlu', 'inanoğlu', 'insanoğul', 'insanoğulu']},
  {'error_word': 'ne',
   'start': 18,
   'end': 19,
   'replacements': ['de',
    've',
    'ne',
    'öne',
    'net',
    'new',
    'e',
    'ye',
    'be',
    'le',
    'n

In [6]:
grammar_correction('dikicinin', lang=None)

{'corrected_text': 'Daničini',
 'number_errors': 1,
 'errors': [{'error_word': 'dikicinin',
   'start': 0,
   'end': 8,
   'replacements': ['Daničini',
    'Daničinih',
    'Daničinim',
    'Doričini',
    'Doričinih',
    'Doričinim',
    'Ivičini',
    'Ivičinih',
    'Ivičinim',
    'Mimičini',
    'Mimičinih',
    'Mimičinim',
    'Miničini',
    'Miničinih',
    'Miničinim',
    'Miričini',
    'Miričinih',
    'Miričinim',
    'damičini',
    'damičinih',
    'damičinim',
    'dekličini',
    'dekličinih',
    'dekličinim',
    'devičini',
    'devičinih',
    'devičinim',
    'dikciji',
    'dirkačičin',
    'lisičini',
    'lisičinih',
    'lisičinim',
    'tičičini',
    'tičičinih',
    'tičičinim']}]}